In [1]:
# -*- coding: utf-8 -*-
# Установка необходимых пакетов
!pip install -q vllm langchain langchain-community faiss-gpu beautifulsoup4 requests pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 MB 6.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 77.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 95.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 39.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 87.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.2 MB/

In [2]:
# необходимые импорты
import subprocess
import time
import torch
import requests
from bs4 import BeautifulSoup
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
from langchain.llms import VLLMOpenAI
from langchain.prompts import PromptTemplate

# Проверка наличия GPU
print(f"GPU available: {torch.cuda.is_available()}")
print(f"GPU count: {torch.cuda.device_count()}")
print(f"GPU name: {torch.cuda.get_device_name(0)}")




GPU available: True
GPU count: 2
GPU name: Tesla T4


# Первые попытки запуска через langchain

In [3]:
# from langchain_community.llms import VLLM
# llm = VLLM(
#     model="Qwen/Qwen-7B-Chat",
#     trust_remote_code=True,
#     tensor_parallel_size=2,
#     gpu_memory_utilization=0.85,
#     max_model_len=2048,
#     max_new_tokens=1024,
#     temperature=0.7,
#     top_p=0.9,
#     stop_token_ids=[151643, 151644, 151645],  # These might need to be adjusted for Qwen
#     skip_special_tokens=True  # This is important to remove special tokens
# )

# # Initialize embedding model
# embedding_model = HuggingFaceEmbeddings(
#     model_name="sentence-transformers/all-MiniLM-L6-v2",
#     model_kwargs={"device": "cuda:1"}  # Use second card for embeddings
)

INFO 03-20 14:57:41 [__init__.py:256] Automatically detected platform cuda.


config.json:   0%|          | 0.00/911 [00:00<?, ?B/s]

configuration_qwen.py:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-7B-Chat:
- configuration_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


INFO 03-20 14:57:44 [config.py:2595] Downcasting torch.float32 to torch.float16.
INFO 03-20 14:57:56 [config.py:583] This model supports multiple tasks: {'generate', 'embed', 'reward', 'score', 'classify'}. Defaulting to 'generate'.
WARNING 03-20 14:57:56 [arg_utils.py:1765] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 03-20 14:57:56 [config.py:1515] Defaulting to use mp for distributed inference
INFO 03-20 14:57:56 [llm_engine.py:241] Initializing a V0 LLM engine (v0.8.1) with config: model='Qwen/Qwen-7B-Chat', speculative_config=None, tokenizer='Qwen/Qwen-7B-Chat', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, dec

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Здесь увидел, что в ответе присутствуют токены конца и начала

In [5]:
# # Running a simple query
# print(llm.invoke("What are the most popular Halloween Costumes?"))

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.12s/it, est. speed input: 1.76 toks/s, output: 24.61 toks/s]

<|im_end|>
<|im_start|>'t think<|im_end|>
<|im_start|>


As an AI language model, I don't have access to real-time data on the popularity of Halloween costumes. However, based on past trends, some of the most popular Halloween costumes include:

1. Witch
2. Superhero
3. Vampire
4. Zombie
5. Ghost
6. Cat or Dog
7. Princess or Prince
8. Harry Potter
9. Star Wars character
10. Elf

These are just a few examples, and the popularity of costumes can vary from year to year based on current trends and popular culture.<|im_end|>



# Тут ради интереса пробовал подход запуска черезе Subprocess

In [6]:

# # 1. Setup vLLM with Qwen on T4x2
# MODEL_ID = "Qwen/Qwen-7B-Chat"  # Suitable for T4x2, use Qwen-1.8B for smaller model if needed

# # Launch vLLM server via subprocess to run in background
# print("Starting vLLM server in background mode...")
# vllm_log = open("vllm_server.log", "w")
# vllm_server = subprocess.Popen(
#     [
#         "python", 
#         "-m", 
#         "vllm.entrypoints.openai.api_server",
#         "--model", MODEL_ID,
#         "--tensor-parallel-size", "2",  # Use both T4 cards
#         "--gpu-memory-utilization", "0.85",  # Optimal GPU memory usage
#         "--max-model-len", "2048",  # Context limitation to save memory
#         "--port", "8000",
#         "--trust-remote-code" 
#     ],
#     stdout=vllm_log,
#     stderr=vllm_log
# )

# print("Waiting for server initialization...")
# time.sleep(120)  # Give the server time to initialize

# # Check if server is running
# try:
#     health_check = requests.get("http://localhost:8000/health")
#     if health_check.status_code == 200:
#         print("vLLM server successfully started!")
#     else:
#         print("Server started but there are issues. Check the log file.")
# except:
#     print("Server not ready yet. Might need more time to initialize.")
#     print("Check log file: vllm_server.log")

Starting vLLM server in background mode...
Waiting for server initialization...
Server not ready yet. Might need more time to initialize.
Check log file: vllm_server.log


In [63]:
# !ps aux | grep vllm

root        2182  0.0  0.0   7376  3328 pts/0    Ss+  14:45   0:00 /bin/bash -c ps aux | grep vllm
root        2184  0.0  0.0   6484  2432 pts/0    S+   14:45   0:00 grep vllm


In [35]:
# # Initialize LLM
# llm = VLLMOpenAI(
#     openai_api_key="EMPTY",
#     openai_api_base="http://localhost:8000/v1",
#     model_name="Qwen/Qwen-7B-Chat",
#     temperature=0.7,
#     max_tokens=1024,
#     top_p =0.9,
#     model_kwargs={
#         "top_k": 40
#     }
# )

# # Initialize embedding model
# embedding_model = HuggingFaceEmbeddings(
#     model_name="sentence-transformers/all-MiniLM-L6-v2",  # Lightweight model suitable for T4
#     model_kwargs={"device": "cuda:1"}  # Use second card for embeddings
# )

# Сбор данных для RAG

In [3]:
# Инициализация модели эмбеддингов
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cuda:1"}  # Используем вторую карту для эмбеддингов
)

# Function to get champion statistics data
def get_champion_stats():
    url = "https://www.metasrc.com/lol/stats"
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(response.content, "html.parser") 
    table = soup.select_one('#table-scroll table')
    rows = table.find_all('tr')
    champions = []
    
    for row in rows:
        cells = row.find_all('td')
        data = [cell.get_text(strip=True) for cell in cells]
        if data:  # Skip empty lists
            # Fix for duplicate champion names
            champion_name = data[0]
            # Check if the name is duplicated (same text repeated)
            if len(champion_name) % 2 == 0:
                half_length = len(champion_name) // 2
                first_half = champion_name[:half_length]
                second_half = champion_name[half_length:]
                # If both halves are identical, use just one
                if first_half == second_half:
                    champion_name = first_half
                    
            champion = {
                "Championname": champion_name,
                "Lane": data[1],
                "Tier": data[2],
                "Score": data[3],
                "Trend": data[4],
                "Winrate": data[5],
                "Rolerate": data[6],
                "PickRate": data[7],
                "BanRate": data[8],
                "KDA": data[9]
            }
            champions.append(champion)
    
    return champions

# Convert champion stats to documents for vector store
def create_stat_documents(champions_data):
    documents = []
    for champ in champions_data:
        text = f"Champion: {champ['Championname']}\n" + \
               f"Lane: {champ['Lane']}\n" + \
               f"Tier: {champ['Tier']}\n" + \
               f"Score: {champ['Score']}\n" + \
               f"Trend: {champ['Trend']}\n" + \
               f"Winrate: {champ['Winrate']}\n" + \
               f"Role rate: {champ['Rolerate']}\n" + \
               f"Pick rate: {champ['PickRate']}\n" + \
               f"Ban rate: {champ['BanRate']}\n" + \
               f"KDA: {champ['KDA']}\n\n"

        if "God" in champ["Tier"] or "S+" in champ["Tier"]:
            text += "This champion is the best and strongest pick for this lane in the current meta.\n"
        elif "S" in champ["Tier"] or "A+" in champ["Tier"]:
            text += "This champion is a strong pick for this lane in the current meta.\n"
        elif "A" in champ["Tier"] or "B+" in champ["Tier"]:
            text += "This champion is a good pick for this lane in the current meta.\n"
        elif "B" in champ["Tier"] or "C+" in champ["Tier"]:
            text += "This champion is an average pick for this lane in the current meta.\n"
        else:
            text += "This champion is a below average pick for this lane in the current meta.\n"
        
        documents.append(Document(
            page_content=text,
            metadata={
                "champion": champ['Championname'],
                "lane": champ['Lane'],
                "tier": champ['Tier'],
                "content_type": "statistics"
            }
        ))
    
    return documents

# List of URLs with guides
guide_urls = [
    "https://www.mobafire.com/league-of-legends/build/25-05-the-bible-of-jungling-shaco-white-crows-sexy-guide-327054",
    "https://www.mobafire.com/league-of-legends/build/25-s1-5-fundamentals-of-aatrox-diamond-guide-remastered-632525",
    "https://www.mobafire.com/league-of-legends/build/25-05-shoks-rank-1-challenger-ahri-guide-635065"
]

# Dictionary to determine champion by URL
champion_map = {
    "327054": "Shaco",
    "632525": "Aatrox",
    "635065": "Ahri"
}

# Function to extract guide ID from URL
def extract_guide_id(url):
    import re
    match = re.search(r'-(\d+)$', url)
    if match:
        return match.group(1)
    return "unknown"

# First, get champion statistics
print("Fetching champion statistics...")
champion_stats = get_champion_stats()
stat_documents = create_stat_documents(champion_stats)
print(f"Created {len(stat_documents)} documents from champion statistics")

# Load data from all guides
all_chunks = []
all_chunks.extend(stat_documents)  # Add statistics documents

for url in guide_urls:
    try:
        print(f"Loading guide: {url}")
        loader = WebBaseLoader(url)
        documents = loader.load()
        
        # Determine champion by URL
        guide_id = extract_guide_id(url)
        champion_name = champion_map.get(guide_id, "Unknown")
        
        # Add champion metadata
        for doc in documents:
            doc.metadata["champion"] = champion_name
            doc.metadata["guide_url"] = url
            doc.metadata["content_type"] = "guide"
        
        # Split into chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            separators=["\n## ", "\n### ", "\n\n", "\n", ".", "?", "!"]
        )
        chunks = text_splitter.split_documents(documents)
        # Analyze chunk content to determine sections
        for chunk in chunks:
            text_lower = chunk.page_content.lower()
            
            # Determine guide sections
            if any(term in text_lower for term in ["item", "build"]):
                chunk.metadata["section"] = "items"
            elif any(term in text_lower for term in ["rune", "runes"]):
                chunk.metadata["section"] = "runes"
            elif any(term in text_lower for term in ["skill", "ability", "abilities"]):
                chunk.metadata["section"] = "skills"
            elif any(term in text_lower for term in ["matchup", "match-up", "versus", "vs", "counter"]):
                chunk.metadata["section"] = "matchups"
            else:
                chunk.metadata["section"] = "general"
        
        print(f"Created {len(chunks)} chunks for {champion_name}'s guide")
        all_chunks.extend(chunks)
        
    except Exception as e:
        print(f"Error loading {url}: {e}")

# Create vector store
print(f"Total documents/chunks: {len(all_chunks)}")
vector_store = FAISS.from_documents(all_chunks, embedding_model)

# Create retriever
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)


# Save vector store for future use
vector_store.save_local("lol_combined_vector_store")

print("Vector store successfully created and saved!")



<ipython-input-3-954bd6573e41>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Fetching champion statistics...
Created 256 documents from champion statistics
Loading guide: https://www.mobafire.com/league-of-legends/build/25-05-the-bible-of-jungling-shaco-white-crows-sexy-guide-327054
Created 77 chunks for Shaco's guide
Loading guide: https://www.mobafire.com/league-of-legends/build/25-s1-5-fundamentals-of-aatrox-diamond-guide-remastered-632525
Created 70 chunks for Aatrox's guide
Loading guide: https://www.mobafire.com/league-of-legends/build/25-05-shoks-rank-1-challenger-ahri-guide-635065
Created 41 chunks for Ahri's guide
Total documents/chunks: 444
Vector store successfully created and saved!


# Финальный вариант для QWEN

Оказалось, что в случае с QWEN надо делать свой класс. Пример для работы с Langchain нашел на их официальном сайте

In [4]:
from langchain.vectorstores import FAISS
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import RetrievalQA
from transformers import AutoModelForCausalLM, AutoTokenizer
from abc import ABC
from langchain.llms.base import LLM
from typing import Any, List, Mapping, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun

# Настройка модели Qwen
model_name = "Qwen/Qwen2.5-7B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Обертка для Qwen
class Qwen(LLM, ABC):
    max_token: int = 1024
    temperature: float = 0.7
    top_p: float = 0.9
    history_len: int = 3

    def __init__(self):
        super().__init__()

    @property
    def _llm_type(self) -> str:
        return "Qwen"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        messages = [
            {"role": "system", "content": "You are game expert. You are a helpful assistant for League of Legends players."},
            {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=self.max_token,
            temperature=self.temperature,
            top_p=self.top_p
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        return response

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"max_token": self.max_token,
                "temperature": self.temperature,
                "top_p": self.top_p,
                "history_len": self.history_len}

# Инициализация модели Qwen
llm = Qwen()


# Предположим, что у вас уже есть vector_store (созданный и заполненный)
vector_store = FAISS.load_local('lol_combined_vector_store', embedding_model,allow_dangerous_deserialization=True)

# Создаем retriever
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

# Устанавливаем шаблон промпта
prompt_template = """
Using the following information from League of Legends guides and champion statistics, answer the user's question.
Context:
{context}
Question: {question}
Please provide a detailed answer based only on the provided information.
If there's no information in the context to answer the question, please say so.
If providing statistical information, mention how strong the champion is in the current meta.
If providing guide information, indicate which section of the guide it's from.
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# Создаем QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True, 
    verbose=True  # Покажет больше информации о процессе
)




config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [5]:
# Для тестирования
result = qa_chain.invoke({"query": "What are the best champions for the MID"})
print(result["result"])



> Entering new RetrievalQA chain...

> Finished chain.
Based on the provided information, the best champions for the MID lane in the current meta are Ahri and Zed, both rated as "God / S+" and identified as the best and strongest picks for this lane.

### Ahri
- **Tier:** God / S+
- **Score:** 88.40
- **Winrate:** 52.67%
- **Role rate:** 95.99%
- **Pick rate:** 10.42%
- **Ban rate:** 4.73%
- **KDA:** 2.75

Ahri is highly favored by players and has a significantly high winrate and role rate, making her a top choice for mid-lane. Her high pick rate indicates that she is popular among players, and the low ban rate suggests she is not heavily restricted by teams. Her KDA of 2.75 further emphasizes her ability to perform well in battles.

### Zed
- **Tier:** God / S+
- **Score:** 77.22
- **Winrate:** 49.97%
- **Role rate:** 64.20%
- **Pick rate:** 7.87%
- **Ban rate:** 21.97%
- **KDA:** 2.29

Zed is also considered one of the best champions for the mid-lane, with a high role rate of 64.20

In [7]:
import re
from typing import List, Dict, Tuple
from langchain.vectorstores import FAISS
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import RetrievalQA
from transformers import AutoModelForCausalLM, AutoTokenizer
from abc import ABC
from langchain.llms.base import LLM
from typing import Any, List, Mapping, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Ground truth champions for each lane
GROUND_TRUTH = {
    "MID": ["Ahri", "Mel", "Sylas", "Zed"],
    "TOP": ["Aatrox", "Darius", "Sett"],
    "ADC": ["Caitlyn", "Ezreal", "Jhin", "Jinx"],
    "JUNGLE": ["Darius", "Viego", "Lee Sin"],
    "SUPPORT": ["Karma", "Lulu", "Lux"]
}

# Queries for each lane
QUERIES = {
    "MID": "What are the best champions for MID?",
    "TOP": "What are the best champions for TOP?",
    "ADC": "What are the best champions for ADC?",
    "JUNGLE": "What are the best champions for JUNGLE?",
    "SUPPORT": "What are the best champions for SUPPORT?"
}

def extract_champions(response: str) -> List[str]:
    """
    Extremely simple function to check if known champions from our ground truth are mentioned in the response.
    Simply checks for string containment without any regex.
    """
    # Combine all possible champions from the ground truth
    all_possible_champions = []
    for champions in GROUND_TRUTH.values():
        for champion in champions:
            if champion not in all_possible_champions:
                all_possible_champions.append(champion)
    
    found_champions = []
    
    # Convert response to lowercase for case-insensitive matching
    response_lower = response.lower()
    
    for champion in all_possible_champions:
        # Simple string containment check
        if champion.lower() in response_lower:
            print(champion)
            print(response_lower)
            found_champions.append(champion)
    
    return found_champions


def calculate_accuracy(extracted_champions: List[str], ground_truth: List[str]) -> Tuple[float, float, float]:
    """
    Calculate precision, recall and F1 score.
    
    Precision = TP / (TP + FP) - What percentage of recommended champions are correct
    Recall = TP / (TP + FN) - What percentage of correct champions were recommended
    F1 = 2 * (Precision * Recall) / (Precision + Recall)
    """
    # Convert to lowercase for case-insensitive matching
    extracted_lower = [champ.lower() for champ in extracted_champions]
    ground_truth_lower = [champ.lower() for champ in ground_truth]
    
    true_positives = sum(1 for champ in extracted_lower if champ in ground_truth_lower)
    false_positives = len(extracted_lower) - true_positives
    false_negatives = sum(1 for champ in ground_truth_lower if champ not in extracted_lower)
    
    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    
    f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
    
    return precision, recall, f1

def get_true_positive_champions(extracted_champions: List[str], ground_truth: List[str]) -> List[str]:
    """Return the list of correctly identified champions."""
    extracted_lower = [champ.lower() for champ in extracted_champions]
    
    true_positives = []
    for idx, champ_lower in enumerate(extracted_lower):
        if champ_lower in [gt.lower() for gt in ground_truth]:
            true_positives.append(extracted_champions[idx])
            
    return true_positives

def evaluate_rag_system(qa_chain, queries: Dict[str, str], ground_truth: Dict[str, List[str]]):
    """
    Evaluate the RAG system's performance across different lane queries.
    """
    results = {}
    overall_precision = 0
    overall_recall = 0
    overall_f1 = 0
    
    print("\n--- League of Legends Champion Recommendation Evaluation ---\n")
    
    for lane, query in queries.items():
        print(f"Testing query: {query}")
        
        # Get response from the RAG system
        response = qa_chain.invoke({"query": query})
        result_text = response["result"]
        print(result_text)
        
        # Extract champions from the response
        extracted_champions = extract_champions(result_text)
        print(extracted_champions)
        
        # Find true positive champions
        true_positive_champions = get_true_positive_champions(extracted_champions, ground_truth[lane])
        
        # Calculate metrics
        precision, recall, f1 = calculate_accuracy(extracted_champions, ground_truth[lane])
        
        # Store results
        results[lane] = {
            "query": query,
            "extracted_champions": extracted_champions,
            "ground_truth": ground_truth[lane],
            "true_positives": true_positive_champions,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "response": result_text
        }
        
        # Print lane results
        print(f"\nLane: {lane}")
        print(f"Extracted champions: {', '.join(extracted_champions)}")
        print(f"Ground truth: {', '.join(ground_truth[lane])}")
        print(f"Correctly identified: {', '.join(true_positive_champions)}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print(f"F1 Score: {f1:.2f}")
        print("-" * 50)
        
        overall_precision += precision
        overall_recall += recall
        overall_f1 += f1
    
    # Calculate overall metrics
    num_lanes = len(queries)
    overall_precision /= num_lanes
    overall_recall /= num_lanes
    overall_f1 /= num_lanes
    
    # Print overall results
    print("\n--- Overall Results ---")
    print(f"Average Precision: {overall_precision:.2f}")
    print(f"Average Recall: {overall_recall:.2f}")
    print(f"Average F1 Score: {overall_f1:.2f}")
    
    # Create and save visualization
    visualize_results(results)
    
    return results

def visualize_results(results):
    """Create visualizations of the evaluation results."""
    # Prepare data for plotting
    lanes = list(results.keys())
    precision_values = [results[lane]["precision"] for lane in lanes]
    recall_values = [results[lane]["recall"] for lane in lanes]
    f1_values = [results[lane]["f1"] for lane in lanes]
    
    # Create DataFrame for easier plotting
    df = pd.DataFrame({
        "Lane": lanes + lanes + lanes,
        "Metric": ["Precision"] * len(lanes) + ["Recall"] * len(lanes) + ["F1 Score"] * len(lanes),
        "Value": precision_values + recall_values + f1_values
    })
    
    # Set up the plot
    plt.figure(figsize=(12, 8))
    
    # Create grouped bar chart
    sns.barplot(x="Lane", y="Value", hue="Metric", data=df)
    
    # Add labels and title
    plt.title("RAG System Evaluation Metrics by Lane", fontsize=16)
    plt.xlabel("Lane", fontsize=14)
    plt.ylabel("Score", fontsize=14)
    plt.ylim(0, 1.0)
    
    # Add score labels on top of bars
    for i, bar in enumerate(plt.gca().patches):
        plt.gca().text(
            bar.get_x() + bar.get_width()/2,
            bar.get_height() + 0.02,
            f"{bar.get_height():.2f}",
            ha="center",
            fontsize=9
        )
    
    # Improve layout
    plt.tight_layout()
    plt.savefig("lol_rag_evaluation_results.png")
    plt.close()
    
    # Create a champion recommendation accuracy plot
    plt.figure(figsize=(14, 10))
    
    for i, lane in enumerate(lanes):
        extracted = results[lane]["extracted_champions"]
        ground_truth = results[lane]["ground_truth"]
        
        plt.subplot(3, 2, i+1)
        
        # Create sets for Venn diagram-like visualization
        extracted_set = set([champ.lower() for champ in extracted])
        truth_set = set([champ.lower() for champ in ground_truth])
        
        # Calculate intersection and differences
        correct = extracted_set.intersection(truth_set)
        missed = truth_set - extracted_set
        incorrect = extracted_set - truth_set
        
        # Create bar chart
        categories = ['Correct', 'Missed', 'Incorrect']
        values = [len(correct), len(missed), len(incorrect)]
        colors = ['green', 'orange', 'red']
        
        bars = plt.bar(categories, values, color=colors)
        
        # Add champion names as text
        correct_list = [champ for champ in extracted if champ.lower() in correct]
        missed_list = [champ for champ in ground_truth if champ.lower() in missed]
        incorrect_list = [champ for champ in extracted if champ.lower() in incorrect]
        
        # Add labels above bars with champion names
        plt.text(0, values[0] + 0.1, ', '.join(correct_list), ha='center', wrap=True)
        if len(missed_list) > 0:
            plt.text(1, values[1] + 0.1, ', '.join(missed_list), ha='center', wrap=True)
        if len(incorrect_list) > 0:
            plt.text(2, values[2] + 0.1, ', '.join(incorrect_list), ha='center', wrap=True)
        
        plt.title(f"{lane} Lane Champion Recommendations")
        plt.ylim(0, max(values) + 2)  # Add space for text
    
    plt.tight_layout()
    plt.savefig("lol_champion_accuracy.png")
    plt.close()

# Main function to run the evaluation
def evaluate_direct_llm(llm, queries: Dict[str, str], ground_truth: Dict[str, List[str]]):
    """
    Evaluate the LLM's performance without RAG (direct invocation) across different lane queries.
    """
    results = {}
    overall_precision = 0
    overall_recall = 0
    overall_f1 = 0
    
    print("\n--- Direct LLM (No RAG) Champion Recommendation Evaluation ---\n")
    
    for lane, query in queries.items():
        print(f"Testing query: {query}")
        
        # Direct LLM call without retrieval
        system_prompt = "You are a game expert and a helpful assistant for League of Legends players. Please recommend the best champions for the lane or role mentioned in the question."
        full_prompt = f"{system_prompt}\n\nQuestion: {query}\n\nPlease provide a detailed answer with specific champion recommendations."
        
        # Get response from the LLM
        result_text = llm.invoke(full_prompt)
        
        # Extract champions from the response
        extracted_champions = extract_champions(result_text)
        
        # Find true positive champions
        true_positive_champions = get_true_positive_champions(extracted_champions, ground_truth[lane])
        
        # Calculate metrics
        precision, recall, f1 = calculate_accuracy(extracted_champions, ground_truth[lane])
        
        # Store results
        results[lane] = {
            "query": query,
            "extracted_champions": extracted_champions,
            "ground_truth": ground_truth[lane],
            "true_positives": true_positive_champions,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "response": result_text
        }
        
        # Print lane results
        print(f"\nLane: {lane}")
        print(f"Extracted champions: {', '.join(extracted_champions)}")
        print(f"Ground truth: {', '.join(ground_truth[lane])}")
        print(f"Correctly identified: {', '.join(true_positive_champions)}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print(f"F1 Score: {f1:.2f}")
        print("-" * 50)
        
        overall_precision += precision
        overall_recall += recall
        overall_f1 += f1
    
    # Calculate overall metrics
    num_lanes = len(queries)
    overall_precision /= num_lanes
    overall_recall /= num_lanes
    overall_f1 /= num_lanes
    
    # Print overall results
    print("\n--- Overall Direct LLM Results ---")
    print(f"Average Precision: {overall_precision:.2f}")
    print(f"Average Recall: {overall_recall:.2f}")
    print(f"Average F1 Score: {overall_f1:.2f}")
    
    # Create and save visualization
    visualize_direct_llm_results(results)
    
    return results

def visualize_direct_llm_results(results):
    """Create visualizations of the direct LLM evaluation results."""
    # Same as visualize_results but with different filenames
    # Prepare data for plotting
    lanes = list(results.keys())
    precision_values = [results[lane]["precision"] for lane in lanes]
    recall_values = [results[lane]["recall"] for lane in lanes]
    f1_values = [results[lane]["f1"] for lane in lanes]
    
    # Create DataFrame for easier plotting
    df = pd.DataFrame({
        "Lane": lanes + lanes + lanes,
        "Metric": ["Precision"] * len(lanes) + ["Recall"] * len(lanes) + ["F1 Score"] * len(lanes),
        "Value": precision_values + recall_values + f1_values
    })
    
    # Set up the plot
    plt.figure(figsize=(12, 8))
    
    # Create grouped bar chart
    sns.barplot(x="Lane", y="Value", hue="Metric", data=df)
    
    # Add labels and title
    plt.title("Direct LLM (No RAG) Evaluation Metrics by Lane", fontsize=16)
    plt.xlabel("Lane", fontsize=14)
    plt.ylabel("Score", fontsize=14)
    plt.ylim(0, 1.0)
    
    # Add score labels on top of bars
    for i, bar in enumerate(plt.gca().patches):
        plt.gca().text(
            bar.get_x() + bar.get_width()/2,
            bar.get_height() + 0.02,
            f"{bar.get_height():.2f}",
            ha="center",
            fontsize=9
        )
    
    # Improve layout
    plt.tight_layout()
    plt.savefig("lol_direct_llm_evaluation_results.png")
    plt.close()
    
    # Create a champion recommendation accuracy plot
    plt.figure(figsize=(14, 10))
    
    for i, lane in enumerate(lanes):
        extracted = results[lane]["extracted_champions"]
        ground_truth = results[lane]["ground_truth"]
        
        plt.subplot(3, 2, i+1)
        
        # Create sets for Venn diagram-like visualization
        extracted_set = set([champ.lower() for champ in extracted])
        truth_set = set([champ.lower() for champ in ground_truth])
        
        # Calculate intersection and differences
        correct = extracted_set.intersection(truth_set)
        missed = truth_set - extracted_set
        incorrect = extracted_set - truth_set
        
        # Create bar chart
        categories = ['Correct', 'Missed', 'Incorrect']
        values = [len(correct), len(missed), len(incorrect)]
        colors = ['green', 'orange', 'red']
        
        bars = plt.bar(categories, values, color=colors)
        
        # Add champion names as text
        correct_list = [champ for champ in extracted if champ.lower() in correct]
        missed_list = [champ for champ in ground_truth if champ.lower() in missed]
        incorrect_list = [champ for champ in extracted if champ.lower() in incorrect]
        
        # Add labels above bars with champion names
        plt.text(0, values[0] + 0.1, ', '.join(correct_list), ha='center', wrap=True)
        if len(missed_list) > 0:
            plt.text(1, values[1] + 0.1, ', '.join(missed_list), ha='center', wrap=True)
        if len(incorrect_list) > 0:
            plt.text(2, values[2] + 0.1, ', '.join(incorrect_list), ha='center', wrap=True)
        
        plt.title(f"{lane} Lane Champion Recommendations")
        plt.ylim(0, max(values) + 2)  # Add space for text
    
    plt.tight_layout()
    plt.savefig("lol_direct_llm_champion_accuracy.png")
    plt.close()

def save_comparison_results(rag_results, direct_llm_results):
    """Create and save visualizations comparing RAG vs Direct LLM performance."""
    lanes = list(rag_results.keys())
    
    # Create comparison metrics dataframe
    comparison_data = []
    
    for lane in lanes:
        comparison_data.extend([
            {"Lane": lane, "Metric": "Precision", "System": "RAG", "Value": rag_results[lane]["precision"]},
            {"Lane": lane, "Metric": "Recall", "System": "RAG", "Value": rag_results[lane]["recall"]},
            {"Lane": lane, "Metric": "F1", "System": "RAG", "Value": rag_results[lane]["f1"]},
            {"Lane": lane, "Metric": "Precision", "System": "Direct LLM", "Value": direct_llm_results[lane]["precision"]},
            {"Lane": lane, "Metric": "Recall", "System": "Direct LLM", "Value": direct_llm_results[lane]["recall"]},
            {"Lane": lane, "Metric": "F1", "System": "Direct LLM", "Value": direct_llm_results[lane]["f1"]}
        ])
    
    df = pd.DataFrame(comparison_data)
    
    # Create comparison visualizations
    plt.figure(figsize=(15, 10))
    
    for i, metric in enumerate(["Precision", "Recall", "F1"]):
        plt.subplot(1, 3, i+1)
        metric_df = df[df["Metric"] == metric]
        
        ax = sns.barplot(x="Lane", y="Value", hue="System", data=metric_df)
        
        plt.title(f"{metric} Comparison: RAG vs Direct LLM", fontsize=14)
        plt.xlabel("Lane", fontsize=12)
        plt.ylabel(f"{metric} Score", fontsize=12)
        plt.ylim(0, 1.0)
        
        # Add value labels on bars
        for i, bar in enumerate(ax.patches):
            ax.text(
                bar.get_x() + bar.get_width()/2,
                bar.get_height() + 0.02,
                f"{bar.get_height():.2f}",
                ha="center",
                fontsize=8
            )
    
    plt.tight_layout()
    plt.savefig("lol_rag_vs_direct_comparison.png")
    plt.close()
    
    # Save detailed comparison to CSV
    comparison_rows = []
    
    for lane in lanes:
        rag_correct = set([x.lower() for x in get_true_positive_champions(
            rag_results[lane]["extracted_champions"], 
            rag_results[lane]["ground_truth"]
        )])
        
        direct_correct = set([x.lower() for x in get_true_positive_champions(
            direct_llm_results[lane]["extracted_champions"], 
            direct_llm_results[lane]["ground_truth"]
        )])
        
        # Unique to RAG
        rag_only = rag_correct - direct_correct
        # Unique to Direct LLM
        direct_only = direct_correct - rag_correct
        # Common to both
        common = rag_correct.intersection(direct_correct)
        
        row = {
            "Lane": lane,
            "RAG Precision": rag_results[lane]["precision"],
            "Direct LLM Precision": direct_llm_results[lane]["precision"],
            "RAG Recall": rag_results[lane]["recall"],
            "Direct LLM Recall": direct_llm_results[lane]["recall"],
            "RAG F1": rag_results[lane]["f1"],
            "Direct LLM F1": direct_llm_results[lane]["f1"],
            "RAG Champions": ', '.join(rag_results[lane]["extracted_champions"]),
            "Direct LLM Champions": ', '.join(direct_llm_results[lane]["extracted_champions"]),
            "Correctly Identified by Both": ', '.join([c for c in common]),
            "Correctly Identified Only by RAG": ', '.join([c for c in rag_only]),
            "Correctly Identified Only by Direct LLM": ', '.join([c for c in direct_only])
        }
        
        comparison_rows.append(row)
    
    # Calculate average improvements
    avg_precision_improvement = sum([rag_results[lane]["precision"] - direct_llm_results[lane]["precision"] for lane in lanes]) / len(lanes)
    avg_recall_improvement = sum([rag_results[lane]["recall"] - direct_llm_results[lane]["recall"] for lane in lanes]) / len(lanes)
    avg_f1_improvement = sum([rag_results[lane]["f1"] - direct_llm_results[lane]["f1"] for lane in lanes]) / len(lanes)
    
    # Add summary row
    summary_row = {
        "Lane": "AVERAGE",
        "RAG Precision": sum([rag_results[lane]["precision"] for lane in lanes]) / len(lanes),
        "Direct LLM Precision": sum([direct_llm_results[lane]["precision"] for lane in lanes]) / len(lanes),
        "RAG Recall": sum([rag_results[lane]["recall"] for lane in lanes]) / len(lanes),
        "Direct LLM Recall": sum([direct_llm_results[lane]["recall"] for lane in lanes]) / len(lanes),
        "RAG F1": sum([rag_results[lane]["f1"] for lane in lanes]) / len(lanes),
        "Direct LLM F1": sum([direct_llm_results[lane]["f1"] for lane in lanes]) / len(lanes),
        "RAG Champions": "",
        "Direct LLM Champions": "",
        "Correctly Identified by Both": f"Precision Improvement: {avg_precision_improvement:.2f}",
        "Correctly Identified Only by RAG": f"Recall Improvement: {avg_recall_improvement:.2f}",
        "Correctly Identified Only by Direct LLM": f"F1 Improvement: {avg_f1_improvement:.2f}"
    }
    
    comparison_rows.append(summary_row)
    
    # Create DataFrame and save to CSV
    comparison_df = pd.DataFrame(comparison_rows)
    comparison_df.to_csv("lol_rag_vs_direct_comparison.csv", index=False)
    print("Comparison results saved to lol_rag_vs_direct_comparison.csv")
    
    return comparison_df

def main():
    print("\n=== Starting RAG System Evaluation ===\n")
    # Run RAG evaluation
    rag_results = evaluate_rag_system(qa_chain, QUERIES, GROUND_TRUTH)
    
    print("\n=== Starting Direct LLM Evaluation (No RAG) ===\n")
    # Run direct LLM evaluation (without RAG)
    direct_llm_results = evaluate_direct_llm(llm, QUERIES, GROUND_TRUTH)
    
    # Save detailed RAG results
    save_detailed_results(rag_results)
    
    # Create and save comparison visualizations and data
    comparison_df = save_comparison_results(rag_results, direct_llm_results)
    
    # Print key findings
    print("\n=== Key Performance Comparison: RAG vs Direct LLM ===\n")
    
    # Get the average row from comparison DataFrame
    avg_row = comparison_df[comparison_df["Lane"] == "AVERAGE"].iloc[0]
    
    # Calculate absolute improvements
    precision_improvement = avg_row["RAG Precision"] - avg_row["Direct LLM Precision"]
    recall_improvement = avg_row["RAG Recall"] - avg_row["Direct LLM Recall"]
    f1_improvement = avg_row["RAG F1"] - avg_row["Direct LLM F1"]
    
    # Calculate relative improvements (percentage)
    if avg_row["Direct LLM Precision"] > 0:
        precision_pct = (precision_improvement / avg_row["Direct LLM Precision"]) * 100
    else:
        precision_pct = float('inf')
        
    if avg_row["Direct LLM Recall"] > 0:
        recall_pct = (recall_improvement / avg_row["Direct LLM Recall"]) * 100
    else:
        recall_pct = float('inf')
        
    if avg_row["Direct LLM F1"] > 0:
        f1_pct = (f1_improvement / avg_row["Direct LLM F1"]) * 100
    else:
        f1_pct = float('inf')
    
    print(f"Precision: RAG {avg_row['RAG Precision']:.2f} vs Direct LLM {avg_row['Direct LLM Precision']:.2f}")
    print(f"Absolute improvement: {precision_improvement:.2f} ({precision_pct:.1f}%)")
    
    print(f"\nRecall: RAG {avg_row['RAG Recall']:.2f} vs Direct LLM {avg_row['Direct LLM Recall']:.2f}")
    print(f"Absolute improvement: {recall_improvement:.2f} ({recall_pct:.1f}%)")
    
    print(f"\nF1 Score: RAG {avg_row['RAG F1']:.2f} vs Direct LLM {avg_row['Direct LLM F1']:.2f}")
    print(f"Absolute improvement: {f1_improvement:.2f} ({f1_pct:.1f}%)")
    
    print("\nComplete results and visualizations have been saved to CSV files and PNG images.")
    
def save_detailed_results(results):
    """Save detailed results to a CSV file."""
    rows = []
    
    for lane, data in results.items():
        extracted = ', '.join(data["extracted_champions"])
        ground_truth = ', '.join(data["ground_truth"])
        true_positives = ', '.join(data["true_positives"])
        
        row = {
            "Lane": lane,
            "Query": data["query"],
            "Extracted Champions": extracted,
            "Ground Truth Champions": ground_truth,
            "Correctly Identified": true_positives,
            "Precision": data["precision"],
            "Recall": data["recall"],
            "F1 Score": data["f1"]
        }
        
        rows.append(row)
    
    # Create DataFrame and save to CSV
    df = pd.DataFrame(rows)
    df.to_csv("lol_rag_evaluation_results.csv", index=False)
    print("Detailed results saved to lol_rag_evaluation_results.csv")

if __name__ == "__main__":
    main()


=== Starting RAG System Evaluation ===


--- League of Legends Champion Recommendation Evaluation ---

Testing query: What are the best champions for MID?


> Entering new RetrievalQA chain...

> Finished chain.
Based on the provided information, the best champions for the MID lane in the current meta are:

1. **Ahri**:
   - Tier: God / S+
   - Score: 88.40
   - Trend: +9.24 (indicating an increasing popularity)
   - Winrate: 52.67%
   - Role rate: 95.99%
   - Pick rate: 10.42%
   - Ban rate: 4.73%
   - KDA: 2.75
   Ahri is the most dominant choice for the MID lane with a high score, winrate, and pick rate. She is highly favored by players due to her powerful abilities and strong presence in team fights.

2. **Mel**:
   - Tier: God / S+
   - Score: 73.06
   - Trend: -11.92 (indicating a decrease in popularity)
   - Winrate: 47.29%
   - Role rate: 51.21%
   - Pick rate: 5.83%
   - Ban rate: 41.55%
   - KDA: 2.46
   While still a top pick, Mel's popularity has been declining. Despite he

In [7]:
# Для тестирования
result = qa_chain.invoke({"query": "What is the best runes for Ahri?"})
print(result["result"])



> Entering new RetrievalQA chain...

> Finished chain.
Based on the provided information, the best rune set for Ahri appears to be Sorcery, particularly when paired with an AD ganking jungler such as Graves or Kindred. The context indicates that Ahri functions "excellent" or "strong" with these types of junglers, suggesting they complement each other well in terms of gameplay synergy.

The context does not directly mention Sorcery as the best runes for Ahri in all situations, but it highlights that Ahri works exceptionally well with junglers like Graves and Kindred, who would likely use Sorcery runes to maximize their damage output and sustain.

Here's a breakdown:

1. **Graves (AD Ganking Jungler):**
   - Ahri is described as "excellent" with Graves, indicating strong synergy between the two.
   - Graves uses Sorcery as his main rune page, focusing on spell vamp, health, and mana regeneration, which aligns well with Ahri’s playstyle.

2. **Kindred (AD Ganking Jungler):**
   - Simila